In [1]:
import pandas as pd
import sys
 
# adding Folder_2 to the system path
sys.path.insert(0, 'E:/coding/github/line_between_two_coordinates/src')

import map_folium as mf

# data from https://www.kaggle.com/datasets/usdot/flight-delays?resource=download&select=flights.csv

In [2]:
folder = 'E:/coding/github/line_between_two_coordinates/data/'
airlines = pd.read_csv(f'{folder}/airlines.csv')
airports = pd.read_csv(f'{folder}/airports.csv')[['IATA_CODE', 'CITY', 'STATE','COUNTRY', 'LATITUDE', 'LONGITUDE']]
flights = pd.read_csv(f'{folder}/flights.csv', ).head(100)

C:\Users\DERPO\AppData\Local\Temp\ipykernel_23512\3363553961.py:4: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights = pd.read_csv(f'{folder}/flights.csv', ).head(100)


In [3]:
flights_resume = flights[['FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']]

In [29]:
df = flights_resume.join(airports.set_index('IATA_CODE'), on='ORIGIN_AIRPORT')
df = df.rename(columns={ 'CITY':'CITY_ORIGIN', 'STATE':'STATE_ORIGIN','COUNTRY':'COUNTRY_ORIGIN', 'LATITUDE':'LATITUDE_ORIGIN', 'LONGITUDE':'LONGITUDE_ORIGIN' })
df = df.join(airports.set_index('IATA_CODE'), on='DESTINATION_AIRPORT')
df = df.rename(columns={ 'CITY':'CITY_DESTINATION', 'STATE':'STATE_DESTINATION','COUNTRY':'COUNTRY_DESTINATION', 'LATITUDE':'LATITUDE_DESTINATION', 'LONGITUDE':'LONGITUDE_DESTINATION' })
df = df.dropna()

In [22]:
states_ocurrences = df.groupby('STATE_ORIGIN').count()[['FLIGHT_NUMBER']]

In [23]:
states_ocurrences

,FLIGHT_NUMBER
STATE_ORIGIN,
AK,11
AZ,5
CA,13
CO,3
FL,2
HI,4
ID,1
IL,5
IN,1


In [30]:
df = df.loc[df['STATE_ORIGIN'] == 'PR' ]

In [31]:
mf.create_map_origin_destination(df,'LATITUDE_ORIGIN',
                   'LONGITUDE_ORIGIN' ,
                    'LATITUDE_DESTINATION' ,
                    'LONGITUDE_DESTINATION',5,0 )

In [5]:
df.head(2)

,x_origin,y_origin,x_destination,y_destination
0,-16.7573,-49.4412,-63.6437,-48.0735
1,-18.4831,-47.3916,-13.6391,-48.3743


In [5]:
# ## IF NEEDS TRANSFORMATION
# transformer = Transformer.from_crs("epsg:2154", "epsg:4326")
# latlong_origin = list(zip(*transformer.transform(df["x_origin"], df["y_origin"])))
# latlong_destination = list(zip(*transformer.transform(df["x_destination"], df["y_destination"])))
# m = folium.Map(latlong_origin[0], zoom_start=15)

In [8]:
latlong_origin = list(zip(df["x_origin"], df["y_origin"]))
latlong_destination = list(zip(df["x_destination"], df["y_destination"]))
m = folium.Map(latlong_origin[0], zoom_start=5)

for origin, destination in zip(latlong_origin, latlong_destination):
    folium.CircleMarker([origin[0], origin[1]],
                        radius=15,
                        fill_color="#3db7e4", # divvy color
                       ).add_to(m)

    folium.CircleMarker([destination[0], destination[1]],
                        radius=15,
                        fill_color="red", # divvy color
                       ).add_to(m)

    folium.PolyLine([[origin[0], origin[1]], 
                     [destination[0], destination[1]]]).add_to(m)
m